# [E-04] 작사가 인공지능 만들기

* 루브릭 평가기준
    * 텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?
    * 특수문자 제거, 토크나이저 생성, 패딩 처리 등의 과정이 빠짐없이 진행되었는가?
    * 텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?

### 시퀀스

* 수열을 영어로 시퀀스라고 함.
* 값이 연속적으로 이어진 자료형들을 총칭하여 '시퀀스 자료형'이라고 한다.
* 예측을 위해 나열된 자료들이 어느 정도 연관성이 있어야 한다.
* 인공지능이 문장을 구성하기 위해 통계에 기반한 방법을 사용한다.
    * **인공지능에게 많은 글을 읽게 함으로써 글을 이해하게 한다.**
    * 즉, 많은 데이터가 곧 좋은 결과를 만들어낸다.(단어를 적재적소에 활용하는 능력이 발달된다.)

### 순환신경망(RNN)

* 'start' 토큰을 맨 앞에 추가해서 문장을 생성 하라는 신호를 준다.
* 생성한 단어를 다시 입력으로 사용하고, 다음 단어를 생성. -> 순환하는 형태
* 인공지능이 문장을 다 만들었으면, 'end' 토큰을 생성.
* 'start' 토큰이 문장의 시작에 더해진 입력 데이터(문제)와 'end' 토큰이 문장 끝에 더해진 출력 데이터(답)이 필요하다.

### 언어 모델

어떤 문구 뒤에 다음 단어가 나올 확률이 높다는 것은 그 단어가 나오는 것이 보다 자연스럽다는 뜻이다.
그렇다고해서 '나는' 뒤에 '밥을'이 나오는 게 자연스럽다는 말은 아님. 단지 '나는' 뒤에 올 수 있는 자연스러운 단어의 경우의 수가 너무 많아 불확실성이 높다.

* n-1개의 단어 시퀀스 w1~wn-1이 주어졌을 때, n번째 단어 wn으로 뭐가 올지 예측하는 확률 모델을 **언어 모델**이라고 한다.

* RNN은 w1,...,wn-1이 주어졌을 때 wn으로 뭐가 올지 예측하는 구조를 가지고 있다.
    * 어떤 텍스트도 언어 모델의 학습 데이터가 될 수 있다.
    * **n-1번째까지의 단어 시퀀스가 x_train, n번째 단어가 y_train이 된다.**

### 데이터 다운로드 및 읽어오기

In [1]:
import os, re
import numpy as np
import tensorflow as tf
import glob

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [2]:
# 중복 파일 제거
import itertools

def check_dup(file1, file2):
    txt1 = []
    txt2 = []
    with open(file1, "r", encoding="utf-8") as f:
        raw = f.read().splitlines()
        txt1.extend(raw)
    with open(file2, "r", encoding="utf-8") as f:
        raw = f.read().splitlines()
        txt2.extend(raw)
    txt1 = set(txt1)
    txt2 = set(txt2)
    diff = txt1.difference(txt2)
    return len(txt1) * 0.05 > len(diff)


for a, b in itertools.combinations(txt_list, 2):
    if check_dup(a, b):
        print(a, b)
        txt_list.remove(b)

/aiffel/aiffel/lyricist/data/lyrics/notorious_big.txt /aiffel/aiffel/lyricist/data/lyrics/notorious-big.txt
/aiffel/aiffel/lyricist/data/lyrics/Kanye_West.txt /aiffel/aiffel/lyricist/data/lyrics/kanye-west.txt


* 지훈님의 중복 파일 제거 코드입니다.

In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue # 길이가 0인 문장은 건너뛴다.
    if sentence[-1] == ':': continue # 문장의 끝이 :인 문장은 건너뛴다.
     
    # 출력되는 양이 너무 많아서 주석 처리
    #print(sentence)

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() #1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) #2
    sentence = re.sub(r'[" "]+', " ", sentence) #3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) #4
    sentence = sentence.strip() #5
    sentence = '<start> ' + sentence + ' <end>' #6
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [5]:
# 정제된 단어를 모을 변수
corpus = []

for sentence in raw_corpus:
    #원하지 않는 문장은 건너 뛴다.
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    #정제 후 담아줌
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [6]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=15000, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)   

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                           padding='post',
                                                          maxlen = 15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1013    3]
 [  37   15 9049 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f17605aaa90>


In [7]:
print(tensor[:3, :10])

[[   2   50    5   91  297   65   57    9  969 6042]
 [   2   17 2639  873    4    8   11 6043    6  329]
 [   2   36    7   37   15  164  282   28  299    4]]


In [8]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [9]:
src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0]
[  50    5   91  297   65   57    9  969 6042    3    0    0    0    0]


In [10]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

### stpe 4. 평가 데이터셋 분리 

In [11]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                         test_size=0.2)

In [12]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


### 인공지능 만들기

In [13]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
lyricist= TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [14]:
for src_sample, tgt_sample in dataset.take(1): break

lyricist(src_sample)

<tf.Tensor: shape=(256, 14, 15001), dtype=float32, numpy=
array([[[ 8.46021503e-05,  2.52009748e-04,  4.61594245e-05, ...,
          2.33305516e-04, -7.78254835e-05, -3.36763012e-04],
        [ 2.90806842e-04,  1.93902131e-04, -6.48359928e-05, ...,
          6.25596789e-04, -2.22000817e-04, -5.48156037e-04],
        [ 2.07553079e-04, -9.14896955e-05,  1.41962417e-04, ...,
          8.35243263e-04, -1.83566342e-04, -6.53463241e-04],
        ...,
        [-4.73332533e-04, -6.98791759e-04,  1.10949168e-03, ...,
          1.26563871e-04,  9.19126964e-04, -5.34387771e-04],
        [-4.29378764e-04, -4.50320076e-04,  1.05934707e-03, ...,
          8.65775946e-05,  8.96921207e-04, -1.50823689e-04],
        [-3.94241972e-04, -4.98914218e-04,  8.54750164e-04, ...,
          1.70037543e-04,  9.50952235e-04,  2.42470463e-07]],

       [[-1.42095509e-04,  9.68535824e-06,  6.28962734e-05, ...,
          6.63788596e-05,  8.19904744e-05,  2.84427224e-04],
        [-1.35691647e-04, -4.09511631e-05, -9

In [15]:
lyricist.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3840256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  15376025  
Total params: 32,855,961
Trainable params: 32,855,961
Non-trainable params: 0
_________________________________________________________________


In [16]:
optimizer = tf.keras.optimizers.Adam()
#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

lyricist.compile(loss=loss, optimizer=optimizer)
lyricist.fit(dataset, epochs=15)

Epoch 1/15
686/686 [==============================] - 131s 187ms/step - loss: 3.6521
Epoch 2/15
686/686 [==============================] - 131s 190ms/step - loss: 3.1615
Epoch 3/15
686/686 [==============================] - 131s 191ms/step - loss: 2.9680
Epoch 4/15
686/686 [==============================] - 131s 190ms/step - loss: 2.8196
Epoch 5/15
686/686 [==============================] - 131s 191ms/step - loss: 2.6910
Epoch 6/15
686/686 [==============================] - 131s 191ms/step - loss: 2.5744
Epoch 7/15
686/686 [==============================] - 131s 191ms/step - loss: 2.4685
Epoch 8/15
686/686 [==============================] - 132s 191ms/step - loss: 2.3691
Epoch 9/15
686/686 [==============================] - 131s 190ms/step - loss: 2.2760
Epoch 10/15
686/686 [==============================] - 131s 191ms/step - loss: 2.1856
Epoch 11/15
686/686 [==============================] - 131s 190ms/step - loss: 2.0991
Epoch 12/15
686/686 [==============================] - 131s 191

In [19]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        predict = model(test_tensor) 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""

    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [20]:
lyric = generate_text(lyricist, tokenizer, init_sentence="<start> she", max_len=20)
print(lyric)

<start> she s got me runnin round and round <end> 


## 마무리

* 텍스트 제너레이션 결과에 대해.
    * 가사 내용은 그럴 듯하게 생성된 듯 하다. 처음엔 10000개 단어를 기억할 수 있는 tokenizer를 만들었는데, 이 결과와 15000개로 늘렸을 때의 결과는 달랐다. 물론 가사 내용이 변한 것에 새로 학습을 한 것도 있고, 다른 원인이 있을 것이라 생각되지만 tokenizer 수치 변경에도 이유가 있을 것이라 생각한다.

* 특수문자 제거
    * 문제 없이 제거된 듯 하다. 이 부분에서 좀 답답했는데, 데이터 수가 많아서 눈으로 확인이 어려웠다. 특수 문자를 찾는 코드에 대한 이해가 부족하다고 생각됨.

* 토크나이저 생성, 패딩 처리
   * tokenizer에 대한 기본 개념은 어렴풋이 이해가 되지만 이걸 처음부터 혼자 작성한다고 하면 못할 것 같다. 문장을 토큰 단위로 쪼개는 건 알겠고, 학습을 위한 것이라는 것도 알겠는데 학습의 메커니즘이 잘 이해가 가지 않았다. 패딩 또한 같은 맥락으로 문장의 길이를 맞춰주기 위함이라는 건 알지만 근본적인 부분이 이해가 안된다고 해야하나..

* 모델의 validation loss
    * 최종 loss는 1.7241로 루브릭 평가기준이었던 2.2보다는 나은 수치를 기록했다. 최대 토큰 개수 15개 이상인 데이터를 학습에서 제외하기 위해 패딩 max_len을 15로 제한했었는데, 물론 이 방법도 맞게 적용한 건지는 아직 모르지만 이 때문인건지, 아니면 epoch를 충분히 줘서인지 처음부터 2.2 이하의 loss값이 나와서 어려운 문제였는지 잘 모르겠다.  

* 최종적으로
    * cv에 비해 가시적으로 바로 볼 수 있는 분야가 아니라는 것을 느꼈다. 아무것도 모르고 4학년 1학기 졸업과제로 했던 영어 리뷰 요약을 어떻게 수행했던 건지 의아할 정도였는데 물론 cv가 쉽다는 건 아니지만 그래도 난 cv가 훨씬 좋은 것 같다. 영어와 한국어를 비롯한 다양한 언어가 존재하고, 관용적인 말도 많은 걸 고려하면 자연어처리 분야는 지금 나한테는 너무 멀게 느껴지는 분야인 듯함...